In [2]:
import requests
import re
from bs4 import BeautifulSoup, Tag
from concurrent.futures import ThreadPoolExecutor, as_completed
import random
import time

from pprint import pprint

import html2text
from lxml import html

In [3]:
def scrape_resume_markdown(url: str, session=None) -> str:
    """Fetch a resume page and convert main content to Markdown-like text."""
    if session is None:
        session = requests.Session()

    response = session.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "lxml")

    # find main content blocks
    target_divs = soup.find_all("div", class_=["media-body", "single-post-body"])
    if not target_divs:
        return ""

    markdown_parts = []

    for div in target_divs:
        div_html = str(div)
        tree = html.fromstring(div_html)

        for element in tree.iter():
            tag = element.tag.lower()

            # headings
            if tag in ["h1", "h2", "h3", "u", "strong", "b"]:
                text = element.text_content().strip()
                if text:
                    markdown_parts.append(f"\n# {text}\n")

            # paragraphs
            elif tag == "p":
                text = element.text_content().strip()
                if text:
                    markdown_parts.append(f"{text}\n")

            # unordered lists
            elif tag == "ul":
                for li in element:
                    li_text = li.text_content().strip()
                    if li_text:
                        markdown_parts.append(f"- {li_text}\n")

            # ordered lists
            elif tag == "ol":
                for i, li in enumerate(element, 1):
                    li_text = li.text_content().strip()
                    if li_text:
                        markdown_parts.append(f"{i}. {li_text}\n")

    markdown_text = "".join(markdown_parts)
    return markdown_text.strip()

In [4]:
markdown_resume = scrape_resume_markdown(url=test_urls[4])
print(markdown_resume)

NameError: name 'test_urls' is not defined

In [1]:
test_urls = [
    "https://www.hireitpeople.com/resume-database/80-peoplesoft-resumes/627367-peoplesoft-hcm-hrms-functional-consultant-resume-3",
    "https://www.hireitpeople.com/resume-database/80-peoplesoft-resumes/613216-peoplesoft-functional-consultant-business-analyst-resume-atlanta-ga-4",
    "https://www.hireitpeople.com/resume-database/69-help-desk-support-resumes/614271-help-desk-support-trading-floor-support-resume-new-york-ny",
    "https://www.hireitpeople.com/resume-database/69-help-desk-support-resumes/606776-computer-technician-field-associate-resume-new-york-ny-11",
    "https://www.hireitpeople.com/resume-database/63-net-developers-architects-resumes/442-net-developeranalystarchitect-resume-",
]

In [5]:
def scrape_resume(url: str, session=None) -> dict:
    """Scrape details from an individual resume page and return plain text."""
    if session is None:
        session = requests.Session()

    response = session.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "lxml")
    data = {}

    target_divs = soup.find_all("div", class_=["single-post-body"])
    print(target_divs)
    text_blocks = []
    for div in target_divs:
        text = div.get_text(separator=" ", strip=True)
        if text:
            text_blocks.append(text)

    clean_text = "\n\n".join(text_blocks)

    return data


In [6]:
data = scrape_resume(url=test_urls[0])
# print(data)

NameError: name 'test_urls' is not defined

In [ ]:
def make_list(ul):
    if isinstance(ul, str):
        soup = BeautifulSoup(ul, "lxml")
        ul_tag = soup.find("ul")
        if not ul_tag:
            return []
    elif isinstance(ul, Tag):
        ul_tag = ul
    else:
        return []

    points = []
    for li in ul_tag.find_all("li", recursive=False):
        text = li.get_text(" ", strip=True)
        if text:
            points.append(text)
    return points


In [7]:
def scrape_resume(url: str, session=None) -> dict:
    """Scrape details from an individual resume page, keeping DOM tags without duplication."""
    if session is None:
        session = requests.Session()

    response = session.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "lxml")
    sections = {}

    target_divs = soup.find_all("div", class_=["media-body", "single-post-body"])
    print(target_divs)
    def get_real_keys(strong_tags):
        possible_keys = {"TECHNICAL SKILLS", "SKILLS", "PROFESSIONAL EXPERIENCE", "EXPERIENCE", "SUMMARY", "OBJECTIVE"}
        ret_list = []
        for tag in strong_tags:
            for possible_key in possible_keys:
                if possible_key in tag:
                    ret_list.append(tag)
        return ret_list

    for div in target_divs:
        real_keys = div.find_all("u")
        # strong_tags = div.find_all("strong")
        # real_keys = get_real_keys(strong_tags)
        print(real_keys)
        for i, u_tag in enumerate(real_keys):
            key = u_tag.get_text(strip=True).replace(":", "").upper()

            content_tags = []
            for sibling in u_tag.parent.next_siblings:
                if isinstance(sibling, Tag):
                    if sibling.find("u"):
                        break
                    content_tags.append(sibling)

            sections[key] = content_tags

    return sections


res_dict = scrape_resume("https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/628271-teradata-dba-resume-hoffman-estates-il-2")
# for key in res_dict.keys():
#     print(key, res_dict[key], end="\n -------------- \n")

[<div class="media-body">
<div>
<h3>Teradata Dba Resume </h3>
<div class="resume-rated mt-0">
<div class="rating-group">
<input checked="" class="rating__input rating__input--none" disabled="" id="rated-none" readonly="" type="radio" value="0"/>
<label aria-label="1 star" class="rating__label" for="rated-1"><i class="rating__icon rating__icon--star fa fa-star"></i></label>
<input class="rating__input" disabled="" id="rated-1" readonly="" type="radio" value="1"/>
<label aria-label="2 stars" class="rating__label" for="rated-2"><i class="rating__icon rating__icon--star fa fa-star"></i></label>
<input class="rating__input" disabled="" id="rated-2" readonly="" type="radio" value="2"/>
<label aria-label="3 stars" class="rating__label" for="rated-3"><i class="rating__icon rating__icon--star fa fa-star"></i></label>
<input class="rating__input" disabled="" id="rated-3" readonly="" type="radio" value="3"/>
<label aria-label="4 stars" class="rating__label" for="rated-4"><i class="rating__icon ra

In [19]:
import re
from typing import List, Optional
import time, random #noqa
from bs4 import BeautifulSoup
import requests
from pydantic import BaseModel


class Experience(BaseModel):
    job_role: str
    responsibilities: List[str]
    environment: Optional[str] = None


class Resume(BaseModel):
    job_role: str
    professional_summary: List[str]
    technical_skills: List[str]
    experiences: List[Experience]


class PostExtractionResult(BaseModel):
    """Result model for post extraction step."""
    job_role: Optional[str]
    structured_content: List[dict]
    full_text: str
    container_text: str
    missing_excerpt: str
    skipped_blocks: List[str]
    warnings: List[str]
    
def normalize_breaks(soup):
    """Convert <br> tags to newline text nodes so .get_text() uses them."""
    for br in soup.find_all("br"):
        br.replace_with("\n")

def clean_whitespace(text):
    lines = [ln.strip() for ln in text.splitlines()]
    while lines and lines[0] == "":
        lines.pop(0)
    while lines and lines[-1] == "":
        lines.pop()
    return "\n".join([re.sub(r'\s+', ' ', ln) for ln in lines])

def extract_job_role(soup):
    media_bodies = soup.find_all("div", class_=re.compile(r"media-body"))
    if media_bodies:
        media_body = media_bodies[0]
        job_title_tag = media_body.find("h3")
        if job_title_tag:
            job_role = job_title_tag.get_text(strip=True)
            if job_role:
                return job_role
    return None

def extract_post_body_safe(
    url: str,
    target_class = "single-post-body",
    class_regex: Optional[str] = None,
    allow_fallback: bool = True,
    debug: bool = False,
    min_word_threshold: int = 120,
    retries: int = 3,
) -> PostExtractionResult:
    headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64)"}
    
    for attempt in range(retries):
        try:
            with requests.Session() as session:
                session.headers.update(headers)
                resp = session.get(url, timeout=20)
                resp.raise_for_status()
                if len(resp.text) < 1000:
                    raise ValueError("Response too short.")
                break
        except Exception as e:
            if attempt == retries - 1:
                raise
            time.sleep(random.uniform(1, 3))
    soup = BeautifulSoup(resp.text, "html.parser")

    normalize_breaks(soup)

    # Identify container
    container = None
    if target_class:
        container = soup.find("div", class_=target_class)
    if not container and class_regex:
        container = soup.find("div", class_=re.compile(class_regex))
    if not container and allow_fallback:
        divs = soup.find_all("div")
        if divs:
            container = max(divs, key=lambda d: len(d.get_text(strip=True)))
    if not container:
        raise ValueError("Could not find a suitable container.")

    raw_container_text = container.get_text(separator="\n", strip=True)
    container_text = clean_whitespace(raw_container_text)

    structured_content = []
    skipped_blocks = []  # 🔹 Track skipped text blocks

    resume_job_role = extract_job_role(soup)

    # 2️⃣ Handle normal paragraphs and lists not under media-body
    for element in container.find_all(["p", "ul"], recursive=True):
        if element.find_parent("div", class_=re.compile(r"media-body")):
            continue  # already captured above

        if element.name == "p":
            text = clean_whitespace(" ".join(element.stripped_strings))
            if len(text.split()) > min_word_threshold:
                skipped_blocks.append(text[:120] + "...")
                continue
            if text:
                structured_content.append({"type": "p", "text": text})

        elif element.name == "ul":
            items = []
            for li in element.find_all("li", recursive=False):
                li_text = clean_whitespace(" ".join(li.stripped_strings))
                if len(li_text.split()) > min_word_threshold:
                    skipped_blocks.append(li_text[:120] + "...")
                    continue
                if li_text:
                    items.append(li_text)
            if items:
                structured_content.append({"type": "ul", "items": items})

    # Join all paragraph text for convenience
    joined_p = "\n\n".join(
        [b["text"] for b in structured_content if b.get("type") == "p"]
    )

    container_words = len(container_text.split())
    joined_words = len(joined_p.split()) if joined_p else 0

    warnings = []
    missing_excerpt = ""
    if container_words > joined_words + 20:
        temp = container_text
        for block in structured_content:
            if block.get("type") == "p":
                temp = temp.replace(block["text"], "")
            elif block.get("type") == "ul":
                for item in block["items"]:
                    temp = temp.replace(item, "")
            elif "job_role" in block:
                temp = temp.replace(block["job_role"], "")
                for sub in block.get("content", []):
                    if sub.get("type") == "p":
                        temp = temp.replace(sub["text"], "")
                    elif sub.get("type") == "ul":
                        for item in sub["items"]:
                            temp = temp.replace(item, "")
        missing_excerpt = temp.strip()[:800]
        if missing_excerpt:
            warnings.append("Container has additional text not captured by structured tags.")

    if "<script" in resp.text.lower() and (container_words == 0 or joined_words == 0):
        warnings.append("Page might be JS-rendered.")

    if debug:
        print("===== DEBUG INFO =====")
        print("Container classes:", container.get("class"))
        print("Job roles found:", sum(1 for b in structured_content if "job_role" in b))
        print("Paragraphs:", sum(1 for b in structured_content if b.get("type") == "p"))
        print("Lists:", sum(1 for b in structured_content if b.get("type") == "ul"))
        print("Skipped blocks:", len(skipped_blocks))
        print("Warnings:", warnings)
        print("======================")

    return PostExtractionResult(
        job_role=resume_job_role,
        structured_content=structured_content,
        full_text=joined_p,
        container_text=container_text,
        missing_excerpt=missing_excerpt,
        skipped_blocks=skipped_blocks,
        warnings=warnings,
    )


In [20]:
res = extract_post_body_safe(url='https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/628271-teradata-dba-resume-hoffman-estates-il-2')

In [21]:
res.model_dump()

{'job_role': 'Teradata Dba Resume',
 'structured_content': [{'type': 'p', 'text': 'OBJECTIVE'},
  {'type': 'ul',
   'items': ['Position where Database Designing and DBA skills and experience in database designing, administration and Management will add value in today’s challenging business and Information Technology.']},
  {'type': 'p', 'text': 'SUMMARY'},
  {'type': 'ul',
   'items': ['8+ years of experience in IT and Teradata administration, designing database solutions with Architecture.',
    '5+ Years of exclusive Production support and deploy of Teradata Patches, fixtures and DBQL settings.',
    'Experienced in archiving, restoring and recovering data on Teradata using ARC utility and TARA GUI.',
    'Excellent knowledge in moving database objects between Teradata systems and Teradata Database.',
    'Very well trained and experienced in scheduling backups and recovery of the entire EDW databases across various geographical locations for the business continuity and response time

In [235]:
def parse_professional_experience(res_dict):
    if "PROFESSIONAL EXPERIENCE" not in res_dict:
        return []

    items = res_dict["PROFESSIONAL EXPERIENCE"]
    prev_col, curr_col = False, None
    exps_list, exp_dict = [], {}
    i = 0

    while i < len(items):
        item = items[i]

        if item.find("strong"):
            strip_strong = item.get_text(strip=True)
            curr_col = ":" in strip_strong
            if prev_col and not curr_col and exp_dict:
                exps_list.append(exp_dict)
                exp_dict = {}

            exp_len = len(exp_dict)
            if exp_len == 0:
                exp_dict["company_name"] = strip_strong
            elif exp_len == 1:
                exp_dict["job_role"] = strip_strong
            else:
                text_lower = strip_strong.lower()

                if "responsi" in text_lower:
                    if i + 1 < len(items):
                        exp_dict["responsibilities"] = make_list(items[i+1])

                elif "environment" in text_lower[:20]:
                    exp_dict["environment"] = " ".join(strip_strong.split(":")[1:]).strip()

            prev_col = curr_col

        i += 1

    if exp_dict:
        exps_list.append(exp_dict)

    return exps_list


In [236]:
res_dict = scrape_resume(test_urls[0])
# print(res_dict)
print(len(res_dict))
for exp in parse_px(res_dict):
    pprint(exp)

[]
[<u><strong>SUMMARY</strong></u>, <u><strong>TECHNICAL SKILLS</strong></u>, <u><strong>PROFESSIONAL EXPERIENCE</strong></u>]
3
{'company_name': 'Confidential, New jersey',
 'environment': 'Application Release 9.1 Tools 8.48,8.49,Microsoft Office '
                'Tools,MS Project, Windows Server 2003, Microsoft SQL Server.',
 'job_role': 'PeopleSoft HCM / HRMS Functional Consultant',
 'responsibilities': ['Making of the initial project plan with activities by '
                      'phases, key milestones and deliverables according to '
                      'the scope of the RFP.',
                      'Participating in meetings with members of the board '
                      'notifying progress and proposing solutions for some '
                      'problems within the project.',
                      'Co-ordination with multiple teams and integration '
                      'architects, in performing analysis for issues regarding '
                      'speed and stabilit